In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

# update import path
import os, sys
sys.path.insert(1, os.path.join(sys.path[0], '..', 'src'))

import io
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split

# import keras modules
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras import optimizers
from keras import losses

In [ ]:
# read csv
data = pd.read_csv(os.path.join('..', 'data', 'datasets', 'log.csv'), index_col=0)

# make datetime index (could also be done directly in read_csv)
data['datetime'] = pd.to_datetime(data['datetime'])
data = data.set_index('datetime')
data = data.dropna()

print (data.shape)
data.head()

In [ ]:
# show the boat approaching the target course
data['target_angle'].plot()
data['boat_angle'].plot()

In [ ]:
# select features and target
y = data['rudder_angle']
x = data[['angle_of_attack', 'boat_heel', 'boat_speed', 'course_error', 'wind_speed']]

# split test/train sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.10, random_state=42)

# create model
model = Sequential([
    Dense(128, activation='relu', input_shape=(5,)),
    Dense(128, activation='relu'),
    Dense(128, activation='relu'),
    Dense(1)
])

# compile model
model.compile(loss='mse', optimizer=optimizers.Adam(lr=0.0002))
model.summary()

# fit model to training data
model.fit(x_train, y_train, validation_split=0.2, epochs=20)

# show performance
pred_y = model.predict(x_test)
print (mean_absolute_error(pred_y, y_test))

In [ ]:
# save model to file
model.save(os.path.join('..', 'src', 'strategies', 'my_deep_strategy', 'my-keras-model.h5'))